In [56]:
#import necessary libraries
import requests
from requests_html import HTMLSession
from requests_html import AsyncHTMLSession
from bs4 import BeautifulSoup
import pprint
import numpy as np

In [7]:
url1 = 'https://tabs.ultimate-guitar.com/tab/misc-soundtrack/a-star-is-born-shallow-chords-2488086'
url2 = 'https://tabs.ultimate-guitar.com/tab/jeff-buckley/hallelujah-chords-198052'
asession = AsyncHTMLSession() #creates Async object

r = await asession.get(url2,timeout=20) #requests website from server and waits for response
#r.html.render(timeout=20)
site = await r.html.arender(timeout=20) #renders in case there is javascript

soup = BeautifulSoup(r.html.html, features='lxml') #creates BeautifulSoup object which will parse the html
body = soup.find(class_='js-page js-global-wrapper') #finds main body of website which contains the strum patterns
patterns = body.find_all(class_='_2u0o0') #every strum pattern is within a class called '_2u0o0'


In [8]:
patterns[0].contents

[<div class="_31jmH _2xk1x _1yupz _1i6F0"></div>,
 <div class="_31jmH _2xk1x"></div>,
 <div class="_31jmH _2xk1x _1yupz"></div>,
 <div class="_31jmH _2xk1x _3N-at _1i6F0"></div>,
 <div class="_31jmH _2xk1x _1yupz"></div>,
 <div class="_31jmH _2xk1x _3N-at"></div>,
 <div class="_31jmH _2xk1x _1yupz _1i6F0"></div>,
 <div class="_31jmH _2xk1x"></div>,
 <div class="_31jmH _2xk1x _1yupz"></div>,
 <div class="_31jmH _2xk1x _3N-at _1i6F0"></div>,
 <div class="_31jmH _2xk1x _1yupz"></div>,
 <div class="_31jmH _2xk1x _3N-at"></div>]

By cross checking with the website, I've determined that the classes that have '_1yupz' are downstrums, the classes that have '_3N-at' are downstrums, and the classes that have neither of those are beats you don't strum on

In [45]:
print(len(patterns)) #the song I'm testing with has 3 strum patterns so this list should hold three Tags
print(type(patterns)) #confirming that patterns is a list/ResultSet
print(type(patterns[0].children)) #checking the type for the children of the Tag
print(type(next(patterns[0].children))) 
print(type(next(patterns[0].children)['class']))
print(next(patterns[0].children)['class'])#Returns class string as a list


3
<class 'bs4.element.ResultSet'>
<class 'list_iterator'>
<class 'bs4.element.Tag'>
<class 'list'>
['_31jmH', '_2xk1x', '_1yupz', '_1i6F0']


In [69]:
strumPatterns = [] #will hold all strum patterns for song with 1=downstrum, 2=upstrum, 0=skip

for pattern in patterns: #iterate through all strum patterns scraped
    currentPattern = [] #will hold current pattern
    count = 0
    for child in pattern.children: #iterate through all strums within the pattern
        className = child['class'] #get class name of the strum to determine if its upstrum, downstrum, or neither
        
        print(className) #print class name for debugging
        
        if '_1yupz' in className: #if it contains this string, it is an downstrum so append a 1
            currentPattern.append(1)
        elif '_3N-at' in className: #if it contains this string, it is an upstrum so append a 2
            currentPattern.append(2)
        else:   #if it contains this string, it is neither so append a 0
            currentPattern.append(0)
        
    strumPatterns.append(currentPattern) #add current pattern to list of all patterns in song
    print("Integer Pattern:",currentPattern)
    print()
           

['_31jmH', '_2xk1x', '_1yupz', '_1i6F0']
['_31jmH', '_2xk1x']
['_31jmH', '_2xk1x', '_1yupz']
['_31jmH', '_2xk1x', '_3N-at', '_1i6F0']
['_31jmH', '_2xk1x', '_1yupz']
['_31jmH', '_2xk1x', '_3N-at']
['_31jmH', '_2xk1x', '_1yupz', '_1i6F0']
['_31jmH', '_2xk1x']
['_31jmH', '_2xk1x', '_1yupz']
['_31jmH', '_2xk1x', '_3N-at', '_1i6F0']
['_31jmH', '_2xk1x', '_1yupz']
['_31jmH', '_2xk1x', '_3N-at']
Integer Pattern: [1, 0, 1, 2, 1, 2, 1, 0, 1, 2, 1, 2]

['_31jmH', '_2xk1x', '_1yupz']
['_31jmH', '_2xk1x']
['_31jmH', '_2xk1x']
['_31jmH', '_2xk1x', '_1yupz']
['_31jmH', '_2xk1x']
['_31jmH', '_2xk1x', '_3N-at']
['_31jmH', '_2xk1x', '_1yupz']
['_31jmH', '_2xk1x']
['_31jmH', '_2xk1x']
['_31jmH', '_2xk1x', '_1yupz']
['_31jmH', '_2xk1x']
['_31jmH', '_2xk1x', '_3N-at']
Integer Pattern: [1, 0, 0, 1, 0, 2, 1, 0, 0, 1, 0, 2]

['_31jmH', '_2xk1x', '_1yupz']
['_31jmH', '_2xk1x']
['_31jmH', '_2xk1x', '_3N-at']
['_31jmH', '_2xk1x', '_1yupz']
['_31jmH', '_2xk1x']
['_31jmH', '_2xk1x', '_3N-at']
['_31jmH', '_2xk1x',

In [70]:
print(np.array(strumPatterns)) #should give 3x12 array because the song being tested has three strum patterns with 12 strums each

[[1 0 1 2 1 2 1 0 1 2 1 2]
 [1 0 0 1 0 2 1 0 0 1 0 2]
 [1 0 2 1 0 2 1 0 2 1 0 2]]
